In [1]:
import xml.etree.ElementTree as xml

import numpy

import toyplot.html
import toyplot.mark

import logging
logging.basicConfig(level=logging.DEBUG)

toyplot.log.setLevel(logging.DEBUG)

In [2]:
# Custom marks must derive from toyplot.mark.Mark
class Circle(toyplot.mark.Mark):
    def __init__(self, x, y, radius):
        super(Circle, self).__init__()
        self._coordinates = {
            "x": x,
            "y": y,
        }
        self._radius = radius
        
    # Marks must return their domain, i.e. their minimum and maximum values along a given axis.
    def domain(self, axis):
        return numpy.array(self._coordinates[axis]), numpy.array(self._coordinates[axis])
    
    # Marks must optionally return their extents, which are a combination of domain coordinates and
    # range extents relative to those coordinates.  This is so the domain and range of an axis can
    # be adjusted so text, etc. don't fall outside the axes.
    def extents(self, axes):
        coordinates = tuple([numpy.array([self._coordinates[axis]]) for axis in axes])
        extents = (
            numpy.array([-self._radius]),
            numpy.array([self._radius]),
            numpy.array([-self._radius]),
            numpy.array([self._radius]),
            )
        return coordinates, extents

# Custom marks must define a _render() function and register it using dispatch() so that it can be
# called at render time.  Note that _render() is registered for a given combination of coordinate
# system and mark.
@toyplot.html.dispatch(toyplot.coordinates.Cartesian, Circle, toyplot.html._RenderContext)
def _render(axes, mark, context):
    x = axes.project("x", mark._coordinates["x"])
    y = axes.project("y", mark._coordinates["y"])
    xml.SubElement(
        context.parent,
        "circle",
        id=context.get_id(mark),
        cx=repr(x),
        cy=repr(y),
        r=str(mark._radius),
        stroke="black",
        fill="none",
        )

In [3]:
canvas = toyplot.Canvas()
axes = canvas.cartesian()
axes.add_mark(Circle(2, 3, 20))
axes.add_mark(Circle(3, 2, 50));

2.0 2.5 3.0 2.0 2.5 3.0